In [1]:
import os
import urllib.request
import zipfile
import tarfile

In [2]:
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

### Pre-trained된 fastText 모델 다운로드

In [3]:
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "./data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)  

In [4]:
Zip = zipfile.ZipFile("./data/wiki-news-300d-1M.vec.zip")
Zip.extractall("./data/")  # ZIP 압축 해제
Zip.close()  
# "wiki-news-300d-1M.vec"

### Data

In [5]:
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [6]:
tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')  # 압축 해제
tar.close()  
# "aclImdb" 

In [7]:
import glob
import os
import io
import string


# 학습 데이터의 tsv 파일을 작성
f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()
        text = text.replace('\t', " ")
        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()
        text = text.replace('\t', " ")
        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [8]:
# 테스트 데이터 작성
f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()
        text = text.replace('\t', " ")
        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [9]:
import string
import re

# 다음 기호는 스페이스(공백)으로 치환합니다(쉼표, 마침표 제외).
# punctuation은 구두점입니다
print("구두점 문자: ", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 전처리
def preprocessing_text(text):
    # 개행 코드 삭제
    text = re.sub('<br />', '', text)

    # 쉼표, 마침표 이외의 기호를 공백으로 치환
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # 쉼표, 마침표의 전후에 공백 추가
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 띄어쓰기(이번에는 영어 데이터이며, 임시로 공백으로 구분)
def tokenizer_punctuation(text):
    return text.strip().split()

# 전처리 및 띄어쓰기를 포함한 함수 정의
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

# 동작을 확인합니다
print(tokenizer_with_preprocessing('I like cats.'))

구두점 문자:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


### DataLoader

In [10]:
import torchtext
from torchtext.legacy import data

max_length = 256
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

In [11]:
train_val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

print('훈련 및 검증 데이터 수', len(train_val_ds))
print('첫번째 훈련 및 검증 데이터', vars(train_val_ds[0]))

훈련 및 검증 데이터 수 25000
첫번째 훈련 및 검증 데이터 {'Text': ['this', 'is', 'one', 'of', 'the', 'best', 'movies', 'i', 've', 'ever', 'seen', '.', 'it', 'has', 'very', 'good', 'acting', 'by', 'hanks', ',', 'newman', ',', 'and', 'everyone', 'else', '.', 'definitely', 'jude', 'law', 's', 'best', 'performance', '.', 'the', 'cinematography', 'is', 'excellent', ',', 'the', 'editing', 'is', 'about', 'as', 'good', ',', 'and', 'includes', 'a', 'great', 'original', 'score', 'that', 'really', 'fits', 'in', 'with', 'the', 'mood', 'of', 'the', 'movie', '.', 'the', 'production', 'design', 'is', 'also', 'a', 'factor', 'in', 'what', 'makes', 'this', 'movie', 'special', '.', 'to', 'me', ',', 'it', 'takes', 'a', 'lot', 'to', 'beat', 'godfather', ',', 'but', 'the', 'fantastic', 'cinematography', 'displayed', 'wins', 'this', 'contest', '.', 'definitely', 'a', 'best', 'picture', 'nominee', 'in', 'my', 'book', '.'], 'Label': '1'}


In [12]:
import random

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

print('훈련 데이터의 수', len(train_ds))
print('검증 데이터의 수', len(val_ds))
print('첫번째 훈련 데이터', vars(train_ds[0]))

훈련 데이터의 수 20000
검증 데이터의 수 5000
첫번째 훈련 데이터 {'Text': ['this', 'film', 'was', 'really', 'bad', 'whether', 'you', 'take', 'it', 'as', 'a', 'sci', 'fi', 'movie', ',', 'as', 'a', 'horror', 'one', 'or', 'even', 'as', 'a', 'comedy', '.', 'the', 'whole', 'thing', 'is', 'ridiculous', '.', 'the', 'film', 'looks', 'and', 'is', 'definitely', 'cheap', ',', 'the', 'actors', 'have', 'no', 'idea', 'of', 'what', 'acting', 'is', 'and', 'the', 'script', 'shows', 'clearly', 'that', 'it', 'was', 'being', 'made', 'along', 'with', 'the', 'shooting', '.', 'it', 'is', 'obvious', 'that', 'the', 'monster', 'in', 'the', 'closet', 'was', 'added', 'because', 'the', 'living', 'head', 'was', 'not', 'scary', 'at', 'all', 'she', 'was', 'even', 'pretty', 'and', 'they', 'thought', 'they', 'needed', 'something', 'more', 'impressive', 'they', 'failed', 'here', 'too', 'the', 'make', 'up', 'is', 'awful', 'even', 'for', 'the', 'late', '50', 's', ',', 'rather', 'funny', '.', 'the', 'film', 'shows', 'clearly', 'why', 'director',

#### Vocabulary

In [13]:
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')

# 단어 벡터의 내용 확인
print("한 단어를 표현하는 차원 수: ", english_fasttext_vectors.dim)
print("단어 수: ", len(english_fasttext_vectors.itos))

한 단어를 표현하는 차원 수:  300
단어 수:  999994


In [14]:
# Vectorized Vocabulary
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

print(TEXT.vocab.vectors.shape)  # 17916개의 단어가 300차원 벡터로 표현되어 있음
TEXT.vocab.vectors

# vocabulary 단어 순서 확인
# TEXT.vocab.stoi

torch.Size([17887, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0276,  0.0025, -0.1891,  ...,  0.0869, -0.0658, -0.1441],
        [-0.1011, -0.0481,  0.1376,  ...,  0.0959,  0.0863, -0.0949],
        [ 0.0147, -0.0415,  0.0006,  ...,  0.0387, -0.0181, -0.0128]])

In [15]:
train_dl = torchtext.legacy.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.legacy.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.legacy.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)

# 동작 확인
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[   2,   52,    8,  ...,    1,    1,    1],
        [   2,   52,   17,  ...,    1,    1,    1],
        [   2,   14, 1634,  ...,    1,    1,    1],
        ...,
        [   2,   21,    8,  ...,    1,    1,    1],
        [   2,   15,   11,  ...,  134, 2910,    3],
        [   2,   35,   26,  ...,    1,    1,    1]]), tensor([140, 211, 139, 256, 146, 109, 256, 256, 256,  58, 256, 177, 256, 256,
        139, 165, 256, 256, 256, 160, 164, 187, 256, 223]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


In [16]:
import math
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchtext

### Embedding

In [17]:
class Embedder(nn.Module):
    '''개별 단어의 ID에 따라 단어의 벡터 표현 반환'''
    
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        
        # fasttext의 pre-trained된 벡터 표현 이용  
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)  # 역전파에 의해 갱신되지 않도록 가중치 '동결'
    
    def forward(self, x):
        x_vec = self.embeddings(x)  # 단어 수 x 300(Embedding에 의한 분산 표현의 차원)

        return x_vec

In [18]:
! git clone https://github.com/gymoon10/utils.git

fatal: destination path 'utils' already exists and is not an empty directory.


In [19]:
batch = next(iter(train_dl))

# Embedding
net1 = Embedder(TEXT.vocab.vectors)
x = batch.Text[0]
x1 = net1(x)  # 단어를 벡터로

print("입력 텐서 크기: ", x.shape)  # 24개의 sequence는 각각 256개의 단어들로 구성되어 있음
print("Embedding된 출력 텐서 크기: ", x1.shape)  # 256개의 각 단어가 300차원의 분산 표현 벡터로 변환됨

입력 텐서 크기:  torch.Size([24, 256])
Embedding된 출력 텐서 크기:  torch.Size([24, 256, 300])


### Positional Encoding


![image](https://user-images.githubusercontent.com/44194558/150710146-0db579e8-3c76-4965-827e-415b2efabdd7.png)

In [20]:
class PositionalEncoder(nn.Module):
    '''단어의 위치 정보를 표현하는 벡터 (위의 Embedding 차원과 동일한 크기)
       (단어 수, 분산 표현의 차원 수) 크기의 Embedding된 단어 벡터를 입력으로 받음'''
    
    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model  # 개별 단어 벡터의 분산 표현 차원 (fastText : 300)
        pe = torch.zeros(max_seq_len, d_model)  # (단어 수 x 분산 표현의 차원수) - (256, 300)

        #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #pe = pe.to(device)

        for pos in range(max_seq_len):  # 각 단어 별로 처리
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))  # 수식 참고
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * i) / d_model)))

        self.pe = pe.unsqueeze(0)  # (1, 256, 300) - minibatch의 차원 추가
        self.pe.requires_grad = False

    def forward(self, x):
        ret = math.sqrt(self.d_model)*x + self.pe
        
        return ret

In [21]:
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)

x = batch.Text[0]
x1 = net1(x)  # 단어 -> 벡터
x2 = net2(x1)  # Positional encoding

print("Embedding된 입력 텐서 크기: ", x1.shape)
print("위치 정보가 더해진 출력 텐서 크기: ", x2.shape)

Embedding된 입력 텐서 크기:  torch.Size([24, 256, 300])
위치 정보가 더해진 출력 텐서 크기:  torch.Size([24, 256, 300])


### Attention

Scaled-dot (single head)

![image](https://user-images.githubusercontent.com/44194558/150711518-3bdd8fa6-1f3b-4482-b9c2-2cc207385926.png)

In [22]:
class Attention(nn.Module):  

    def __init__(self, d_model=300):
        super().__init__()
        
        # Query Key Value
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        self.out = nn.Linear(d_model, d_model)

        # scale 조정 변수
        self.d_k = d_model

    def forward(self, q, k, v, mask):
        q = self.q_linear(q)  # (24, 256, 300)
        k = self.k_linear(k)
        v = self.v_linear(v)
        
        # Attention value 계산
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)  # (24, 256, 256)

        # Masking
        mask = mask.unsqueeze(1)  # (24, 1, 256)
        weights = weights.masked_fill(mask==0, -1e9)  # mask=0인 부분을 지정된 값으로 채움
        
        # Attention Map
        normalized_weights = F.softmax(weights, dim=-1)
        output = torch.matmul(normalized_weights, v)  # (24, 256, 300)  
        output = self.out(output)

        return output, normalized_weights

### Position Wise Feed-Forward

In [23]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        
        return x

### Transformer

In [24]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        
        # Layer Normalization layer
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        
        # Attention layer
        self.attn = Attention(d_model)
        
        # Position wise Feed-Forward layer
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):  # x : (24, 256, 300) / mask : (24, 256)
        x_normalized = self.norm_1(x)
        output, normalized_weights = self.attn(x_normalized, x_normalized, x_normalized, mask)  # output : (24, 256, 300)  
        
        # skip-conn
        x2 = x + self.dropout_1(output)  # (24, 256, 300)
        
        x_normalized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normalized2))

        return output, normalized_weights

In [25]:
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)

# Mask
x = batch.Text[0]  # (24, 256) - 256개의 단어들에 Attention 연산을 적용할 지 여부 (1:적용 / 0:미적용)
input_pad = 1
input_mask = torch.where(x != input_pad, 1, 0)

x1 = net1(x)  
x2 = net2(x1)  
x3, normalized_weights = net3(x2, input_mask) 

print("입력 텐서 크기: ", x2.shape)
print("출력 텐서 크기: ", x3.shape)
print("Attention 크기: ", normalized_weights.shape)

입력 텐서 크기:  torch.Size([24, 256, 300])
출력 텐서 크기:  torch.Size([24, 256, 300])
Attention 크기:  torch.Size([24, 256, 256])


In [26]:
print('입력 단어 벡터 :')
print(x[1])
print()
print('Mask 벡터:')
print(input_mask[1])

입력 단어 벡터 :
tensor([    2,    12,    11,    28,     4,   175,    19,     4,    85,   110,
           48,  1102,     5,     8,   176,  1220,    80,   259,   525,    42,
         1102,     5,  1026,     4,  1102,   110,    13,   340,     6,     4,
          107,    13,    15,    24,    30,    28,  2183,    46,   872,   188,
           80,     5,    37,    30,   287,    16,    26,   205,    70,  2735,
           13,   134, 13264,     5,    37,    30,    53,  4388,    48,    71,
          461,     7,   421,   174,   525,     7,    71,  1317,    42,  1102,
            5,    15,  2443,    75,    57,  1656,     5,     3,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,    

#### Attention 연산 개요

In [27]:
input = x2  # (24, 256, 300)
mask = input_mask  # (24, 256)
d_model = 300
d_k = 300

q_linear = nn.Linear(d_model, d_model)
v_linear = nn.Linear(d_model, d_model)
k_linear = nn.Linear(d_model, d_model)

k = k_linear(input)  # (24, 256, 300)
q = q_linear(input)
v = v_linear(input)

weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(d_k)  # (24, 256, 256)

mask = mask.unsqueeze(1) # (24, 1, 256)
weights = weights.masked_fill(mask == 0, -1e9) # (24, 256, 256)

normalized_weights = F.softmax(weights, dim=-1)  # (24, 256, 256)
output = torch.matmul(normalized_weights, v)  # (24, 256, 300)

### Classification Head

TransformerBlock 모듈을 반복하여 나온 출력을 입력 받아 분류 예측 수행

In [28]:
class ClassificationHead(nn.Module):
    '''Transformer_Block의 출력을 사용하여 긍정/부정 분류 수행'''

    def __init__(self, d_model=300, output_dim=2):
        super().__init__()

        self.linear = nn.Linear(d_model, output_dim)  
        
        # 가중치 초기화
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):
        x0 = x[:, 0, :]  # 각 미니 배치(문장 sequence)에서 첫 번째 단어([CLS])의 representation추출
        out = self.linear(x0)  # 감성 분류 수행

        return out

In [29]:
batch = next(iter(train_dl))

net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)

x = batch.Text[0]
x1 = net1(x)  # word embedding
x2 = net2(x1)  # add positon embedding
x3, normlized_weights = net3(x2, input_mask)  # Self-Attention
x4 = net4(x3)  # 최종 출력의 0번째 단어를([CLS]) 사용하여, 분류 결과 출력

print("입력 텐서 사이즈: ", x3.shape)
print("출력 텐서 사이즈: ", x4.shape)

입력 텐서 사이즈:  torch.Size([24, 256, 300])
출력 텐서 사이즈:  torch.Size([24, 2])


In [30]:
x4

tensor([[-2.0942,  0.6606],
        [-2.0327,  0.7043],
        [-2.0396,  0.7601],
        [-2.0260,  0.7558],
        [-2.0614,  0.6709],
        [-2.0653,  0.7512],
        [-2.0462,  0.6340],
        [-2.0345,  0.6139],
        [-2.0876,  0.6783],
        [-2.1357,  0.6143],
        [-1.9574,  0.7648],
        [-1.9603,  0.7537],
        [-1.9933,  0.6586],
        [-2.0521,  0.7081],
        [-2.0625,  0.6678],
        [-2.0148,  0.6663],
        [-2.0907,  0.6845],
        [-1.9380,  0.6660],
        [-2.0109,  0.7194],
        [-2.0861,  0.6188],
        [-1.9956,  0.6884],
        [-1.9384,  0.6976],
        [-2.0447,  0.7043],
        [-2.1579,  0.7072]], grad_fn=<AddmmBackward0>)

### 최종 Transformer 모듈 Class

In [31]:
class TransformerClassification(nn.Module):

    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()

        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)


    def forward(self, x, mask):
        x1 = self.net1(x)  
        x2 = self.net2(x1)  
        x3_1, normlized_weights_1 = self.net3_1(x2, mask)  
        x3_2, normlized_weights_2 = self.net3_2(x3_1, mask)  
        x4 = self.net4(x3_2)  

        return x4, normlized_weights_1, normlized_weights_2

In [32]:
# Transformer Module
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

x = batch.Text[0]
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x, input_mask)

print("출력 텐서 크기: ", out.shape)
print("출력 텐서의 sigmoid: ", F.softmax(out, dim=1))

출력 텐서 크기:  torch.Size([24, 2])
출력 텐서의 sigmoid:  tensor([[0.3377, 0.6623],
        [0.3233, 0.6767],
        [0.3259, 0.6741],
        [0.3443, 0.6557],
        [0.3231, 0.6769],
        [0.3467, 0.6533],
        [0.3252, 0.6748],
        [0.3255, 0.6745],
        [0.3040, 0.6960],
        [0.3249, 0.6751],
        [0.3414, 0.6586],
        [0.3187, 0.6813],
        [0.3625, 0.6375],
        [0.2948, 0.7052],
        [0.3440, 0.6560],
        [0.3079, 0.6921],
        [0.3238, 0.6762],
        [0.2876, 0.7124],
        [0.3664, 0.6336],
        [0.3343, 0.6657],
        [0.3550, 0.6450],
        [0.3269, 0.6731],
        [0.3200, 0.6800],
        [0.3329, 0.6671]], grad_fn=<SoftmaxBackward0>)


## Training

gpu 사용에 에러가 발생하여 위의 과정으로 구현한 Transformer 모듈을 import 해서 사용하는 방식으로 학습 진행

https://github.com/YutaroOgawa/pytorch_advanced/tree/master/7_nlp_sentiment_transformer/utils

In [33]:
# Model
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# Init
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# Train
net.train()

# TransformerBlock 초기화
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)

print('네트워크 설정 완료')

네트워크 설정 완료


In [34]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()을 계산한 뒤 nn.NLLLoss(negative log likelihood loss)를 계산

# 최적화 기법 설정
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [35]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    print('-----start-------')

    net.cuda()

    torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  
            else:
                net.eval()   

            epoch_loss = 0.0  # epoch별 total loss
            epoch_corrects = 0  # epoch별 정답의 수 

            for batch in (dataloaders_dict[phase]):  # batch는 Text와 Lable의 사전 오브젝트

                #inputs = batch.Text[0]  # 문장 sequence
                #labels = batch.Label  # 정답 label

                inputs = batch.Text[0].cuda()  # 문장
                labels = batch.Label.cuda()  # 라벨

                # optimizer 초기화
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):

                    # mask 작성
                    input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
                    input_mask = (inputs != input_pad)

                    # minibatch의 문장 sequence, mask를 Transformer에 입력
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # loss

                    _, preds = torch.max(outputs, 1)  # prediction

                    # 학습 시에는 역전파 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  #
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch별 loss와 정답률
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [36]:
dataloaders_dict = {"train": train_dl, "val": val_dl}

In [37]:
num_epochs = 10
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-----start-------


RuntimeError: ignored

#### 구현된 module을 import해서 학습

In [38]:
! git clone https://github.com/gymoon10/utils

fatal: destination path 'utils' already exists and is not an empty directory.


In [39]:
from utils.transformer import TransformerClassification

net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

In [40]:
net.train()

net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)

TransformerBlock(
  (norm_1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
  (norm_2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
  (attn): Attention(
    (q_linear): Linear(in_features=300, out_features=300, bias=True)
    (v_linear): Linear(in_features=300, out_features=300, bias=True)
    (k_linear): Linear(in_features=300, out_features=300, bias=True)
    (out): Linear(in_features=300, out_features=300, bias=True)
  )
  (ff): FeedForward(
    (linear_1): Linear(in_features=300, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear_2): Linear(in_features=1024, out_features=300, bias=True)
  )
  (dropout_1): Dropout(p=0.1, inplace=False)
  (dropout_2): Dropout(p=0.1, inplace=False)
)

In [41]:
criterion = nn.CrossEntropyLoss()

learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [42]:
num_epochs = 10
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.5282 Acc: 0.7212
Epoch 1/10 |  val  |  Loss: 0.3875 Acc: 0.8228
Epoch 2/10 | train |  Loss: 0.3997 Acc: 0.8209
Epoch 2/10 |  val  |  Loss: 0.3582 Acc: 0.8372
Epoch 3/10 | train |  Loss: 0.3747 Acc: 0.8356
Epoch 3/10 |  val  |  Loss: 0.3433 Acc: 0.8534
Epoch 4/10 | train |  Loss: 0.3585 Acc: 0.8430
Epoch 4/10 |  val  |  Loss: 0.3723 Acc: 0.8388
Epoch 5/10 | train |  Loss: 0.3463 Acc: 0.8494
Epoch 5/10 |  val  |  Loss: 0.3253 Acc: 0.8594
Epoch 6/10 | train |  Loss: 0.3423 Acc: 0.8518
Epoch 6/10 |  val  |  Loss: 0.3224 Acc: 0.8624
Epoch 7/10 | train |  Loss: 0.3339 Acc: 0.8557
Epoch 7/10 |  val  |  Loss: 0.3174 Acc: 0.8678
Epoch 8/10 | train |  Loss: 0.3253 Acc: 0.8600
Epoch 8/10 |  val  |  Loss: 0.3173 Acc: 0.8684
Epoch 9/10 | train |  Loss: 0.3173 Acc: 0.8628
Epoch 9/10 |  val  |  Loss: 0.3290 Acc: 0.8626
Epoch 10/10 | train |  Loss: 0.3112 Acc: 0.8676
Epoch 10/10 |  val  |  Loss: 0.3218 Acc: 0.8656


## Test

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   
net_trained.to(device)

epoch_corrects = 0 

for batch in (test_dl):  # test 데이터의 DataLoader

    inputs = batch.Text[0].to(device)  
    labels = batch.Label.to(device)  

    # 순전파(forward) 계산
    with torch.set_grad_enabled(False):

        # mask 작성
        input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
        input_mask = (inputs != input_pad)

        # Transformer에 입력
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # 라벨을 예측

        # 결과 계산
        # 정답수의 합계를 갱신
        epoch_corrects += torch.sum(preds == labels.data)

# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개의 정답률: {:.4f}'.format(len(test_dl.dataset),epoch_acc))

테스트 데이터 25000개의 정답률: 0.8532


In [44]:
# HTML 작성 함수 구현
def highlight(word, attn):
    "Attention 값이 클 때 문자 배경을 진한 빨간색으로 하는 html을 출력하는 함수"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normalized_weights_1, normalized_weights_2, TEXT):
    "HTML 데이터를 작성한다"

    # index에 해당하는 결과 추출
    sentence = batch.Text[0][index]  # Sentence
    label = batch.Label[index]  # Label
    pred = preds[index]  # Prediction

    # index에 해당하는 Attention map 추출
    attens1 = normalized_weights_1[index, 0, :]  # 0번째의 <cls>의 Attention
    attens1 /= attens1.max()

    attens2 = normalized_weights_2[index, 0, :]  # 0번째의 <cls>의 Attention
    attens2 /= attens2.max()

    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 표시용 HTML 작성
    html = '정답 라벨: {}<br>추론 라벨: {}<br><br>'.format(label_str, pred_str)

    # 첫번째 단의 Attention
    html += '[TransformerBlock의 첫번째 단의 Attention을 시각화]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 두번째 단의 Attention
    html += '[TransformerBlock의 두번째 단의 Attention을 시각화]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html

In [45]:
from IPython.display import HTML

# Transformer로 처리

# 미니 배치 준비
batch = next(iter(test_dl))

# GPU가 사용 가능하면 GPU로 데이터를 보낸다
inputs = batch.Text[0].to(device)  # 문장
labels = batch.Label.to(device)  # 라벨

# mask 작성
input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
input_mask = (inputs != input_pad)

# Transformer에 입력
outputs, normalized_weights_1, normalized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # 예측

In [46]:
index = 3  # 출력할 데이터
html_output = mk_html(index, batch, preds, normalized_weights_1,
                      normalized_weights_2, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력

In [53]:
index = 5  # 출력할 데이터
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력

In [51]:
\index = 23  # 출력할 데이터
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력